In [2]:
# Instalamos el driver de cassandra
# sudo pip install cassandra-driver
# sudo pip uninstall cassandra-driver

In [3]:
# Vamos a importar
from cassandra.cluster import Cluster
# Vamos a conectarno a la base de datos:

## Iniciareos el contenedor de Docker con la iagen de cassandra:
'''
docker pull cassandra

Cluster: Para esta práctica solo usaremos un nodo
docker run --name prac2 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

docker run --name prac2 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d imagen-cassandra

docker run --name prac2 -p 192.168.99.100:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d imagen-cassandra

# Propuesta de Alejandro

sudo docker run --name practica2 -p 127.0.0.1:9042:9042 -p 127.0.0.1:9160:9160 -d imagen-cassandra

Conexión:
docker exec -it prac2 bash

Información del Nodo:
nodetool status

'''
# Indicación de nuestra imagen

cluster = Cluster(contact_points=['0.0.0.0'], port=9042)

# Generar la conexión
session = cluster.connect()

In [3]:
'''
Creación del KEYSPACE:  que vamos a llamar peliculas
y le vamos a poner una estrategia de replicación de tipo 
SimpleStrategy y con un factor de replicación igual a 1.
'''
session.execute("""CREATE KEYSPACE base_datos WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};""")


In [4]:
# Seleccionamos el KEYSPACE
session.set_keyspace('base_datos')

### Funciones

In [5]:
import pandas as pd

In [6]:
def detecta_libro(titulo, autor):
  existe = False 
  consulta = list(session.execute('SELECT  id_libro FROM con1 WHERE titulo = ' +  '\'' + titulo + '\'' + ' AND autor = ' +  '\'' + autor +  '\'' + 'ALLOW FILTERING;'))
  
  try:
    id_libro = pd.DataFrame(consulta).iloc[0][0]
    existe = True
  except:
    id_libro = list(session.execute('SELECT MAX(id_libro) FROM con1'))
    id_libro = pd.DataFrame(id_libro).iloc[0][0] + 1 
  return id_libro , existe

In [7]:
# Función para generar nuevos usuarios y registros:

def ingresa_nu(nombre, ap, am, titulo, autor, categoria, calif):
  calif = str(calif)
  membresia = list(session.execute('SELECT MAX(membresia) FROM con1;'))
  membresia = str(pd.DataFrame(membresia).iloc[0][0] + 1)
  id_libro, existe_libro = detecta_libro(titulo, autor)
  id_libro = str(id_libro)
  session.execute('INSERT INTO con1(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')
  session.execute('INSERT INTO con3(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')
  if existe_libro:
    session.execute('INSERT INTO con2(membresia,id_libro,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', '  + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')
  else:
    session.execute('INSERT INTO con2(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')


  print('Membresia:', membresia)

In [8]:
# Funcion para ingresar registros de un usuario ya registrado

def ingresa_m(membresia, titulo, autor, categoria, calif):
  membresia = str(membresia)
  calif = str(calif)
  consulta = list(session.execute('SELECT nombre, a_p, a_m FROM con1 WHERE membresia = ' + membresia + ';' ))
  consulta = pd.DataFrame(consulta)
  nombre = consulta.iloc[0][0]
  ap = consulta.iloc[0][1]
  am = consulta.iloc[0][2]
  id_libro, existe_libro = detecta_libro(titulo, autor)
  id_libro = str(id_libro)
  session.execute('INSERT INTO con1(membresia,id_libro,titulo,autor,categoria,calificacion) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ');')
  session.execute('INSERT INTO con3(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')
  if existe_libro:
    session.execute('INSERT INTO con2(membresia,id_libro,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', '  + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')
  else:
    session.execute('INSERT INTO con2(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (' + membresia + ', ' + id_libro + ', \'' + titulo +  '\', \'' + autor + '\', \'' + categoria + '\', ' + calif + ', \'' + nombre + '\', \'' + ap + '\', \'' + am + '\');')


In [9]:
def ingresa_calif():
  tiene_m = input('¿Tienes membresía? (Escribe Si o No) ')
  if tiene_m == 'Si':
    membresia = input('Ingresa numero de membresia ')
    titulo = input('Ingresa título ')
    autor = input('Ingresa autor ')
    categoria = input('Ingresa categoria ')
    calif = input('Ingresa calificación (Entre 0 y 5) ')
    assert 0 <= float(calif) and float(calif) <= 5, "Ingrese una calificación entre 0 y 5"
    ingresa_m(membresia, titulo, autor, categoria, calif)
  if tiene_m == 'No':
    nombre = input('Ingresa nombre ')
    ap = input('Ingresa apellido paterno ')
    am = input('Ingresa apellido materno ')
    titulo = input('Ingresa título ')
    autor = input('Ingresa autor ')
    categoria = input('Ingresa categoria ')
    calif = input('Ingresa calificación (Entre 0 y 5) ')
    assert 0 <= float(calif) and float(calif) <= 5, "Ingrese una calificación entre 0 y 5"
    ingresa_nu(nombre, ap, am, titulo, autor, categoria, calif)

### Creación de la Tablas.

In [ ]:
# Ingresemos los datos:

session.execute("""
                INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61682,5,'100 años de soledad','Gabriel García Marquez','Realismo Mágico',5.00,'Daniel','Fragoso','Alvarado','Mexico','2000-08-16');
                """)

In [ ]:
'''
Ingresamos los datos:
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (1,5,'100 años de soledad','Gabriel García Marquez','Realismo Mágico',5.00,'Daniel','Fragoso','Alvarado');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,6,'La casa de Bernarda Alba','Federico García Lorca','Tragedia',5.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (3,7,'Una educación','Tara Westover','Ciencia Ficcion',5.00,'Hugo','Rangel','Ramirez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,8,'Do androids dream of electric sheep?','Philip K. Dick','Novela',4.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (3,9,'Solo quedó nuestra historia','Adam Silvera','Cuento',3.00,'Hugo','Rangel','Ramirez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (3,10,'El Tapiz Amarillo','Charlotte Perkins Gilman','Novela',4.00,'Hugo','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,11,'Lazarillo de Tormes',NULL,'Novela',1.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (1,12,'La cancion de Aquiles','Madeline Miller','Novela',5.00,'Daniel','Fragoso','Alvarado');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,13,'Doña Perfecta','Benito Pérez Galdós','Cuento',4.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (3,14,'La metamorfosis','Franz Kafka','Cuento',4.00,'Hugo','Rangel','Ramirez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,15,'Erecciones, eyaculaciones, exhibiciones','Charles Bukowski','Ciencia Ficcion',4.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,16,'Un mundo feliz','Aldous Huxley','Novela',5.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,17,'Ender''s Game','Orson Scott Card','Novela',3.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (1,18,'Pedro Paramo','Juan Rulfo','Novela',3.00,'Daniel','Fragoso','Alvarado');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,19,'Travesuras de la niña mala','Mario Vargas Llosa','Novela',5.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (3,12,'La cancion de Aquiles','Madeline Miller','Novela',4.00,'Hugo','Rangel','Ramirez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (2,20,'El Periquillo de Sarniento','José Joaquín Fernández de Lizardi','Novela',2.00,'Enrique','Gómez','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (4,21,'Exhalation','Ted Chiang','Cuento',4.00,'Fernando','Santa Rita','Vizuet');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (5,22,'1Q84','Haruki Murakami','Novela',5.00,'Guillermo','Andrés','Urbano');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (5,23,'El coronel no tiene quien le escriba','Gabriel García Márquez','Novela',4.00,'Guillermo','Andrés','Urbano');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (5,24,'El señor de las moscas','William Golding','Novela',4.00,'Guillermo','Andrés','Urbano');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (5,25,'La maquina del tiempo','H. G. Wells','Ciencia Ficcion',4.00,'Guillermo','Andrés','Urbano');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (5,26,'La guerra de los mundos','H. G. Wells','Ciencia Ficcion',4.00,'Guillermo','Andrés','Urbano');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,27,'Bajo la rueda','Hermann Hesse','Novela',4.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,28,'Siddharta','Hermann Hesse','Novela',5.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,29,'El Lobo Estepario','Hermann Hesse','Novela',5.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,30,'One Shot','Lee Child','Suspenso',4.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,20,'El Periquillo de Sarniento','José Joaquín Fernández de Lizardi','Novela',4.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (7,31,'Harry Potter','J. K. Rowling','Fantasia',5.00,'Yeudiel','Lara','Moreno');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (8,30,'One Shot','Lee Child','Suspenso',5.00,'Eduardo','Garduño','Martinez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (4,33,'On the road','Jack Kerouac','Novela',5.00,'Fernando','Santa Rita','Vizuet');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (9,34,'The Pillars of the Earth','Ken Follett','Novela',4.00,'Edgar','Callejas','Hernandez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,35,'Rayuela','Julio Cortázar','Novela',5.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,36,'Crimen y castigo','Fiódor Dostoyevski','Novela',4.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,37,'Fenomenología del espíritu','Georg Wilhelm Friedrich Hegel','Ensayo',4.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,38,'Ulises','James Joyce','Novela',4.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,39,'Siddharta','Hermann Hesse','Novela',3.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (6,16,'Un mundo feliz','Aldous Huxley','Ciencia Ficcion',4.00,'Fernando','Avitúa','Varela');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,40,'The catcher in the rye','J. D. Salinger','Novela',5.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (10,41,'La insoportable levedad del ser','Milan Kundera','Novela',4.00,'Ariel','Fuentes','Cruz');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (11,40,'The catcher in the rye','J. D. Salinger','Novela',4.00,'Lucas','Cervantes','Beutelspacher');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (11,42,'The masque of the red death','Edgar Allan Poe','Cuento',5.00,'Lucas','Cervantes','Beutelspacher');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (11,43,'The tell-tale heart','Edgar Allan Poe','Cuento',5.00,'Lucas','Cervantes','Beutelspacher');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (11,18,'Pedro Paramo','Juan Rulfo','Novela',2.00,'Lucas','Cervantes','Beutelspacher');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (11,44,'The fall of the house of usher','Edgar Allan Poe','Cuento',4.00,'Lucas','Cervantes','Beutelspacher');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,40,'The catcher in the rye','J. D. Salinger','Novela',5.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,45,'Dune','Frank Herbert','Ciencia Ficcion',4.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,46,'El psicoanalista','John Katzenbach','Thriller',4.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,47,'Othello','William Shakespeare','Teatro',4.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,48,'El principito','Antoine de Saint-Exupéry','Cuento',5.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,49,'El asesinato de pitágoras','Marcos Chicot','Thriller histórico',4.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (12,50,'Hábitos Atómicos','James Clear','Autoayuda',5.00,'Marcela','Cruz','Larios');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,51,'Los Recuerdos del Porvenir','Elena Garro','Realismo mágico',5.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (14,52,'Pride and Prejudice','Jane Austen','Novela',5.00,'Camila','Silva','Velazquez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,53,'Testimonios sobre Mariana','Elena Garro','Realismo mágico',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (14,54,'The Lord of the Rings','J. R. R. Tolkien','Novela',4.00,'Camila','Silva','Velazquez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,55,'Neverwhere','Neil Gaiman','Fantasia Oscura',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,56,'Valis','Philip K. Dick','Ciencia Ficcion',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (14,57,'La Historia del Rey Transparente','Rosa Montero','Novela',5.00,'Camila','Silva','Velazquez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,58,'La semana de colores','Elena Garro','Cuento',9.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (14,59,'¡Adiós, Cordera!','Leopoldo Alas','Cuento',5.00,'Camila','Silva','Velazquez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,60,'Temporada de Huracanes','Fernanda Melchor','Novela',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,61,'The Sirens of Titan','Kurt Vonnegut','Ciencia Ficcion',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,62,'Un hogar solido','Elena Garro','Teatro',5.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (14,63,'La Gitanilla','Miguel de Cervantes','Novela',4.00,'Camila','Silva','Velazquez');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (13,64,'Ser y tiempo','Martin Heidegger','Ensayo',4.00,'Laz',NULL,NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,18,'Pedro Paramo','Juan Rulfo','Novela',4.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,16,'Un mundo feliz','Aldous Huxley','Ciencia Ficcion',5.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,65,'Crónicas marcianas','Ray Bradbury','Ciencia Ficcion',5.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,66,'1984','George Orwell','Ciencia Ficcion',4.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,67,'Spivak-Calulus','Michael Spivak','Académico',3.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,68,'Física de lo imposible','Michio Kaku','Divulgación',4.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,69,'The Lord of the Rings The Fellowship of the Ring','J. R. R. Tolkien','Fantasia',5.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,70,'The Lord of the Rings The Two Towers','J. R. R. Tolkien','Fantasia',5.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (15,71,'The Lord of the Rings The Return of the King','J. R. R. Tolkien','Fantasia',5.00,'Pablo','Castillo','Del Valle');
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,72,'To Kill A Mockingbird','Harper Lee','Ficción Doméstica',5.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,73,'La Dama de las Camelias','Alejandro Dumas Hijo y Alexandre Dumas','Tragedia',4.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,74,'Farenheit 451','Ray Bradbury','Ficción Distópica',4.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,75,'The Handmaid''s Tale','Margaret Atwood','Ficción Distópica',4.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,76,'La Sombra del Viento','Carlos Ruiz Zafón','Misterio',3.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,77,'Los Hermanos Karamazov','Fiódor Dostoyevski','Novela Filosófica',5.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,78,'Anna Karenina','León Tolstói','Novela Realista',5.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,79,'I Shall Seal the Heavens','Er Gen','Xianxia',3.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (16,80,'The Second Coming of Gluttony','Enrique Fragoso Rangel','Wuxia',4.00,'Alejandro','Leon',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,81,'El Aleph','Jorge Luis Borges','Cuento',5.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,82,'Ficciones','Jorge Luis Borges','Cuento',5.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,18,'Pedro Paramo','Juan Rulfo','Novela',4.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,83,'The Physician','Noah Gordon','Novela',4.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,84,'The Classical Theory of Fields','Lev Landáu','Académico',4.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,85,'Quantum Mechanics: Non-Relativistic Theory','Lev Landáu','Académico',4.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,65,'Crónicas marcianas','Ray Bradbury','Ciencia Ficcion',3.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,45,'Dune','Frank Herbert','Ciencia Ficcion',5.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,86,'Modern Quantum Mechanics','J. J. Sakurai','Académico',4.00,'Bruce','Nolasco',NULL);
INSERT INTO mytable(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m) VALUES (17,8,'Do androids dream of electric sheep?','Philip  K. Dick','Ciencia Ficcion',4.00,'Bruce','Nolasco',NULL);
'''

In [10]:
# Creamos la tabla
session.execute("""
                CREATE TABLE con1 (
                    membresia int,
                    id_libro int,
                    titulo text,
                    autor text, 
                    categoria text,
                    calificacion float,
                    nombre text STATIC,
                    a_p text STATIC,
                    a_m text STATIC,
                    PRIMARY KEY (membresia, categoria, id_libro));
                """)


In [11]:
# Creamos la tabla
session.execute("""
                CREATE TABLE con2 (
                    membresia int,
                    id_libro int,
                    titulo text STATIC,
                    autor text STATIC, 
                    categoria text STATIC,
                    calificacion float,
                    nombre text,
                    a_p text,
                    a_m text,
                    PRIMARY KEY (id_libro, membresia));
                """)

In [12]:
# Creamos la tabla
session.execute("""
                CREATE TABLE con3 (
                    membresia int,
                    id_libro int,
                    titulo text,
                    autor text, 
                    categoria text,
                    calificacion float,
                    nombre text,
                    a_p text,
                    a_m text,
                    PRIMARY KEY (categoria, id_libro, membresia));
                """)

Por la naturaleza de Cassandra vamos a pensar primero en la consulta antes de generar la tabla:

1. La categoría preferida de un cliente dado.

In [49]:
import pandas as pd

In [13]:
def Con1(usuario):
    usuario = str(usuario)
    us = session.execute('SELECT categoria, AVG(calificacion) AS cal FROM con1 WHERE membresia = ' + usuario + ' GROUP BY categoria;')

    df = pd.DataFrame(list(us))

    # Obtengamos aquellas categorias que mejor evaluadas han sido:

    return(df[df.cal == df.cal.max()])

In [24]:
'''
Kike = 2
'''

Con1(2)


,categoria,cal
3,Tragedia,5.0


In [26]:
'''
Bruce = 17
'''
Con1(17)

,categoria,cal
2,Cuento,5.0


In [23]:
'''
Pablo = 15
'''
Con1(15)

,categoria,cal
3,Fantasia,5.0


2. Obtener los clientes que más disfrutaron un libro dado.

In [14]:
def Con2(id_libro):
    id_libro = str(id_libro)
    libro = session.execute('SELECT nombre, a_p, a_m, calificacion FROM con2 WHERE id_libro = ' +  id_libro + ';')

    df = pd.DataFrame(list(libro))

    # Obtengamos el usuario(s)
    return(df[df.calificacion == df.calificacion.max()])

In [31]:
'''
Pedro Paramo : 18
'''

Con2(18)

,nombre,a_p,a_m,calificacion
2,Pablo,Castillo,Del Valle,4.0
3,Bruce,Nolasco,None,4.0


In [32]:
'''
The catcher in the rye : 40
'''

Con2(40)

,nombre,a_p,a_m,calificacion
0,Ariel,Fuentes,Cruz,5.0
2,Marcela,Cruz,Larios,5.0


3. Los mejores libros de una categoría dada.

In [15]:
def Con3(categoria):
    cat = session.execute('SELECT id_libro, titulo,  AVG(calificacion) AS cal FROM con3 WHERE categoria = \'' + categoria + '\' GROUP BY id_libro;')

    df = pd.DataFrame(list(cat))

    # Obtengamos el los mejores
    return(df[df.cal == df.cal.max()])

In [36]:
'''
Novela
'''

Con3('Novela')

,id_libro,titulo,cal
4,16,Un mundo feliz,5.0
7,19,Travesuras de la niña mala,5.0
9,22,1Q84,5.0
13,28,Siddharta,5.0
14,29,El Lobo Estepario,5.0
15,33,On the road,5.0
17,35,Rayuela,5.0
23,52,Pride and Prejudice,5.0
25,57,La Historia del Rey Transparente,5.0


In [37]:
'''
Ciencia Ficcion
'''

Con3('Ciencia Ficcion')

,id_libro,titulo,cal
0,7,Una educación,5.0


In [40]:
'''
Académico 
'''

Con3('Académico')

,id_libro,titulo,cal
1,84,The Classical Theory of Fields,4.0
2,85,Quantum Mechanics: Non-Relativistic Theory,4.0
3,86,Modern Quantum Mechanics,4.0


Definimos una función para optimizar las consultas:

In [11]:
def consulta():
  tipo = input('Ingrese el número según la consulta que desee hacer \n 1: Categoría preferida de un cliente ' +
               '\n 2: Clientes que más difrutaron un libro' +
               '\n 3: Mejores libros de una categoría \n')
  if tipo == '1':
    cliente = input('Ingrese membresia del cliente')
    return Con1(cliente)
  if tipo == '2':
    cliente = input('Ingrese el id del libro')
    return Con2(cliente)
  if tipo == '3':
    categoria = input('Ingrese la categoría')
    return Con3(categoria)

In [16]:
consulta()

Ingrese el número según la consulta que desee hacer 
 1: Categoría preferida de un cliente 
 2: Clientes que más difrutaron un libro
 3: Mejores libros de una categoría 
1
Ingrese membresia del cliente17


,categoria,cal
2,Cuento,5.0


In [17]:
consulta()

Ingrese el número según la consulta que desee hacer 
 1: Categoría preferida de un cliente 
 2: Clientes que más difrutaron un libro
 3: Mejores libros de una categoría 
2
Ingrese el id del libro11


,nombre,a_p,a_m,calificacion
0,Enrique,Gómez,Martinez,1.0


In [18]:
consulta()

Ingrese el número según la consulta que desee hacer 
 1: Categoría preferida de un cliente 
 2: Clientes que más difrutaron un libro
 3: Mejores libros de una categoría 
3
Ingrese la categoríaTeatro


,id_libro,titulo,cal
1,62,Un hogar solido,5.0


### Ejemplo de Ingreso a la base de Datos

In [20]:
ingresa_calif()

¿Tienes membresía? (Escribe Si o No) No
Ingresa nombre Alejandro
Ingresa apellido paterno Pimentel
Ingresa apellido materno Alarcon
Ingresa título Rebelión en la Granja
Ingresa autor George Orwell
Ingresa categoria Novela
Ingresa calificación (Entre 0 y 5) 5
Membresia: 18


In [21]:
ingresa_calif()

¿Tienes membresía? (Escribe Si o No) Si
Ingresa numero de membresia 18
Ingresa título 1984
Ingresa autor George Orwell
Ingresa categoria Novela
Ingresa calificación (Entre 0 y 5) 1
